In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            f1_score,
                            plot_confusion_matrix,
                            precision_recall_curve,
                            precision_score,
                            recall_score)

In [3]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [4]:
mass = pd.read_pickle('../data/03_dummy_df.pickle')

In [22]:
mass.head(2)

,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,notes,...,protester_id_type_political_group,protester_id_type_prisoners,protester_id_type_protestors_generic,protester_id_type_religious_group,protester_id_type_soldiers_veterans,protester_id_type_students_youth,protester_id_type_victims_families,protester_id_type_women,protester_id_type_workers_unions,protester_id_type_nan
0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,...,0,0,0,0,0,0,0,0,1,0
1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,...,0,0,0,0,0,1,0,0,0,0


In [6]:
list(mass.columns)

['id',
 'country',
 'ccode',
 'region',
 'protestnumber',
 'protesterviolence',
 'location',
 'protesteridentity',
 'sources',
 'notes',
 'protester_id_type',
 'partipants_number',
 'protest_size_category',
 'pop_male',
 'pop_female',
 'pop_total',
 'pop_density',
 'prosperity_2020',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'coun

In [7]:
# dummifying protest_size and protester_id_type columns 
mass = pd.get_dummies(mass, columns=['protest_size_category', 'protester_id_type'], dummy_na=True)

Below, attempting a Logistic Regression for each state response, starting with `arrests`.

In [9]:
# X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
#                               'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])
# y = mass['arrests']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

# ss = StandardScaler()

# X_train_sc = ss.fit_transform(X_train)
# X_test_sc = ss.transform(X_test)

# lr = LogisticRegression(solver='lbfgs', max_iter=1000)

# lr.fit(X_train_sc, y_train)

LogisticRegression(max_iter=1000)

In [10]:
# lr.score(X_train_sc, y_train)

0.8637931034482759

In [11]:
# lr.score(X_test_sc, y_test)

0.851888341543514

Now that I have created and evaluated the Logistic Regression model using the training and test set, I will scale the entire `X` set and make predictions for each row in the DataFrame. This will allow me to add two new columns: 
+ **`arrests_predicted`**: column of 0s and 1s indicating whether the protest is predicted to have arrests as one of the state responses. 
+ **`arrests_probability`**: the calculated probability that the protest will have arrests as one of the state responses

In [12]:
# ss = StandardScaler()

# X_sc = ss.fit_transform(X)

In [14]:
# arrests_predicted_all = lr.predict(X_sc)

# arrests_probability_all = lr.predict_proba(X_sc)[:,1]

In [15]:
# mass['arrests_predicted'] = arrests_predicted_all

# mass['arrests_probability'] = arrests_probability_all 

In [17]:
# checking out how it looks 
# mass[['arrests', 'arrests_predicted','arrests_probability']][22:33]

,arrests,arrests_predicted,arrests_probability
22,0,0,0.379786
23,1,1,0.606608
24,0,0,0.209131
25,0,1,0.594836
26,0,0,0.162792
27,0,0,0.392477
28,0,0,0.120245
29,1,1,0.536060
30,1,0,0.472129
31,0,0,0.211884


In [8]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15225 entries, 0 to 15224
Columns: 227 entries, id to protester_id_type_nan
dtypes: datetime64[ns](2), float64(6), int32(180), int64(11), object(6), uint8(22)
memory usage: 13.8+ MB


I will create three functions: 

In [39]:
def evaluate_model(y_true, y_preds):
        
    f1_score(y_true, y_preds)
    
    precision = precision_score(y_true, y_preds)
    
    recall = recall_score(y_true, y_preds)

    return f1_score, precision, recall

In [34]:
def build_confusion_matrix(y_true, y_preds, title):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
    disp = plot_confusion_matrix(model, 
                                 X_test_sc, 
                                 y_test, 
#                                  display_labels=
                                 cmap="Blues")
    
    disp.ax_.set_title(title)
    print(title)
    print(disp.confusion_matrix)
    
    plt.show
    
    return

In [40]:
def state_response_predictor(df, features, target, model):
    
    X = features 
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model.fit(X_train_sc, y_train)
    
    y_preds_train = model.predict(X_train_sc)
    y_preds_test = model.predict(X_test_sc)
    
    f1_score_train, precision_train, recall_train = evaluate_model(y_train, y_preds_train)
    f1_score_test, precision_test, recall_test = evaluate_model(y_test, y_preds_test)
    
    print(f'{target}- F1_score for {model} model, train set: {f1_score_train}')
    print(f'{target}- F1_score for {model} model, test set: {f1_score_test}')
    print(f'{target}- Precision for {model} model, train set: {precision_train}')
    print(f'{target}- Precision for {model} model, test set: {precision_test}')
    print(f'{target}- Recall for {model} model, train set: {recall_train}')
    print(f'{target}- Recall for {model} model, teset set: {recall_test}')

#     build_confusion_matrix(y_test, y_preds_test)

    return model, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test 

In [41]:
def response_prediction_columns(model, df, features):
    
    ss = StandardScaler()
    
    X = features
    X_sc = ss.fit_transform(X)
    
    predicted_column = model.predict(X_sc)
    probability_column = model.predict_proba(X_sc)[:,1]
    
    return predicted_column, probability_column

# ss = StandardScaler()

# X_sc = ss.fit_transform(X)

# arrests_predicted_all = lr.predict(X_sc)

# arrests_probability_all = lr.predict_proba(X_sc)[:,1]

# mass['arrests_predicted'] = arrests_predicted_all

# mass['arrests_probability'] = arrests_probability_all 

_Implementing three above functions for each state response. Will be tweaking the model throughout._

In [42]:
features_1 = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])

LogisticRegression_1 = LogisticRegression(solver='lbfgs', max_iter=1000)

# lr_arrests_1 = state_response_predictor(mass, features_1, 'arrests', lr_1)

In [44]:
lr_arrests_1, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(mass, 
                                                                                                                                   features_1, 
                                                                                                                                   'arrests', 
                                                                                                                                   LogisticRegression_1)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: <function f1_score at 0x7faa678cf160>
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: <function f1_score at 0x7faa678cf160>
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835


In [48]:
(sorted(list(mass.columns)))

['accomodation',
 'arrests',
 'beatings',
 'ccode',
 'country',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus',
 'country_Czech Republic',
 'country_Czechoslovakia',
 'country_Denmark',
 'country_Djibouti',
 'country_Dominican Republic',
 'country_Ecuador',
 'country_Egypt',
 'count

In [50]:
mass['arrests_predicted'], mass['arrests_probability'] = response_prediction_columns(lr_arrests_1, mass, features_1)

In [51]:
mass[['arrests', 'arrests_predicted', 'arrests_probability']][22:33]

,arrests,arrests_predicted,arrests_probability
22,0,0,0.379786
23,1,1,0.606608
24,0,0,0.209131
25,0,1,0.594836
26,0,0,0.162792
27,0,0,0.392477
28,0,0,0.120245
29,1,1,0.536060
30,1,0,0.472129
31,0,0,0.211884


In [43]:
model_performance_dict = {}

# ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
# ['arrests_BC', 'accomodation_BC', 'beatings_BC', 'crowddispersal_BC', 'ignore_BC', 'killings_BC', 'shootings_BC']
 
for classifier in ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']:
    
    model, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(mass, 
                                                                                                                                features_1, 
                                                                                                                                classifier,
                                                                                                                                model)
    model_performance_dist['Classifier'] = classifier
    model_performance_dict['class_balance'] = mass[classifier].value_counts(normalize=True))
    model_performance_dict['model'] = model
    model_performance_dict['f1_score_train'] = f1_score_train
    model_performance_dict['f1_score_test'] = f1_score_test
    model_performance_dict['precision_train'] = precision_train
    model_performance_dict['precision_test'] = precision_test
    model_performance_dict['recall_train'] = recall_train
    model_performance_dict['recall_test'] = recall_test
    
model_performance_df = pd.DataFrame(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: <function f1_score at 0x7faa678cf160>
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: <function f1_score at 0x7faa678cf160>
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835


KeyError: 'classifier'